In [19]:
import boto3
import json

In [20]:
def get_config(filepath='secrets.json'):
    config = {}
    with open(filepath, 'r') as f:
        config = json.load(f)
    return config

In [21]:
region_name = 'us-east-1'
config = get_config()
aws_access_key_id = config['aws_access_key_id']
aws_secret_access_key = config['aws_secret_access_key']

endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

# Uncomment this line to use in production
# endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    endpoint_url=endpoint_url,
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# This will return $10,000.00 in the MTurk Developer Sandbox
print(client.get_account_balance()['AvailableBalance'])

10000.00


# Qualifier

In [41]:
questions = open('screening/qualifier-question.xml', mode='r').read()
answers = open('screening/qualifier-answer-key.xml', mode='r').read()

In [42]:
qual_response = client.create_qualification_type(
                        Name='Hearing Screening Qualifier Test - 2',
                        Keywords='test, qualification, hearing test, listening, screening',
                        Description='This is a brief hearing screening/test',
                        QualificationTypeStatus='Active',
                        RetryDelayInSeconds=60*60, #After 1 hour
                        Test=questions,
                        AnswerKey=answers,
                        TestDurationInSeconds=300)

print(qual_response['QualificationType']['QualificationTypeId'])

3B9TX86P8U75IWXYDPCU6CTT9T9RL1


URL - https://workersandbox.mturk.com/qualifications/36AMEIB6EYUNZ3H0YMXJK1MLZUMVN0

  
URL w/ dropdowns - https://workersandbox.mturk.com/qualifications/3B9TX86P8U75IWXYDPCU6CTT9T9RL1

In [32]:
#   "Title": String,
#   "Description": String,
#   "Reward": String,
#   "AssignmentDurationInSeconds": Integer,
#   "Keywords": String,
#   "AutoApprovalDelayInSeconds": Integer,
#   "QualificationRequirements": QualificationRequirementList

num_approved_hits_atleast_1000={
    'QualificationTypeId': '00000000000000000040',
    'Comparator':'GreaterThanOrEqualTo',
    'IntegerValues':[1000]
}

percent_total_approved_hits={
    'QualificationTypeId': '000000000000000000L0',
    'Comparator':'GreaterThanOrEqualTo',
    'IntegerValues':[97]
}

hearing_screening_qualifier={
    'QualificationTypeId': '36AMEIB6EYUNZ3H0YMXJK1MLZUMVN0',
    'Comparator':'EqualTo',
    'IntegerValues':[100]
}

new_hit_type = client.create_hit_type(
    Title='Audio Perceptual Ordering Task - with screening',
    Description='Audio Perceptual Ordering Task - with screening',
    Reward='1.17',
    AssignmentDurationInSeconds=15*60, #15 mins
    Keywords='audio, listening test, sound evaluation',
    AutoApprovalDelayInSeconds=7*24*60*60, # 7 days
#     QualificationRequirements=[num_approved_hits_atleast_1000,percent_total_approved_hits,hearing_screening_qualifier]
    QualificationRequirements=[hearing_screening_qualifier]
)

In [33]:
new_hit_type

{'HITTypeId': '3J684U0BLUYQAHXUO3902LI651TGGZ',
 'ResponseMetadata': {'RequestId': 'a8c367c9-49c4-4973-b273-0217eb6affe4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a8c367c9-49c4-4973-b273-0217eb6affe4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '46',
   'date': 'Thu, 10 Mar 2022 07:49:47 GMT'},
  'RetryAttempts': 0}}

In [34]:
# From Stackoverflow - https://stackoverflow.com/a/56397390 
class ExternalQuestion:
    """
    An object for constructing an External Question.
    """
    schema_url = "http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd"
    template = '<ExternalQuestion xmlns="%(schema_url)s"><ExternalURL>%%(external_url)s</ExternalURL><FrameHeight>%%(frame_height)s</FrameHeight></ExternalQuestion>' % vars()

    def __init__(self, external_url, frame_height):
        self.external_url = external_url
        self.frame_height = frame_height

    def get_as_params(self, label='ExternalQuestion'):
        return {label: self.get_as_xml()}

    def get_as_xml(self):
        return self.template % vars(self)

In [35]:
question = ExternalQuestion("https://d3uv9uvrvmzlps.cloudfront.net/exp2-2/index-exp2-2-2.html", 0)
question.get_as_xml()

#   "HITTypeId": String,
#   "Question": String,
#   "HITLayoutId": String,
#   "HITLayoutParameters": HITLayoutParameterList,
#   "LifetimeInSeconds": Integer,
#   "MaxAssignments": Integer,
#   "AssignmentReviewPolicy": ReviewPolicy,
#   "HITReviewPolicy": ReviewPolicy,
#   "RequesterAnnotation": String,
#   "UniqueRequestToken": String

new_hit = client.create_hit_with_hit_type(
    HITTypeId="3J684U0BLUYQAHXUO3902LI651TGGZ",
    Question=question.get_as_xml(),
#     LifetimeInSeconds=10*24*60*60, #10 days
    LifetimeInSeconds=1*24*60*60, #1 days
    MaxAssignments=10
)
print("HITID = " + new_hit['HIT']['HITId'])

HITID = 3NC6WP7WKPSYEY4BOLBJQBXSHT0WWK


### List qualification request (both successful and otherwise). Note: Quals created via API dont show correct approved numbers on AMT Admin web page. Use this API for exact numbers. 

In [45]:
list_qual_requests = client.list_workers_with_qualification_type(QualificationTypeId='3B9TX86P8U75IWXYDPCU6CTT9T9RL1')
list_qual_requests